In [1]:
import csv
import pandas as pd 
import os 
import re 
import asyncio
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from functools import partial
from zhipuai import ZhipuAI

# 1. 微博：香港prompt

In [ ]:
data_weibo=pd.read_csv('data_space_score_last_weibo_post.csv')
# 1. 微博：香港ip——sample：1k
data_weibo_hk=data_weibo[data_weibo['IP地址']=='香港']
print(len(data_weibo_hk))
# 2. 微博：非香港ip——sample：1k
data_weibo_gd=data_weibo[data_weibo['IP地址']!='香港']
print(len(data_weibo_gd))
sampled_hkip = data_weibo_hk.sample(n=1000)
sampled_gdip = data_weibo_gd.sample(n=1000)


In [ ]:
client = ZhipuAI(api_key="535af024d07df4663facbf5af957b179.aAxEiUuegF8HcETN")

prompt_learning="""
香港人和广东人、其他大陆人的语言风格、词汇使用、语法结构、话题内容、地理标记不同，请仔细区分。处理香港人文本时，注重港式粤语词汇、特殊用语及英文混杂的特点。处理广东人文本时，注意广州粤语与香港粤语的区别。请分别输出帖子中香港人、广东人概率，输出概率值（0-100）。
例如：
帖子：“請問北上大陆的深圳買计数器和生果，可以過到關回香港嗎，唔該，thanks”。 
分析： “過到關”粤语词汇， “北上”“大陆”“计数器”“生果”港人常用词汇，涉及香港的情况，粤语夹杂了英语；所以香港人概率90%
返回格式：“香港：90%；广东10%；”
"""
# 异步运行同步函数的辅助函数
async def run_in_executor(executor, func, *args):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(executor, func, *args)

# 进行API调用的同步函数
def call_api_sync(client, post_prompt, post_id):
    try:
        glm_completion = client.chat.completions.create(
            model="glm-3-turbo",
            messages=[{"role": "user", "content": post_prompt}],
            temperature=0.5
        )
        response = glm_completion.choices[0].message.content if glm_completion.choices else "No response"
        return (post_id, post_prompt, response)
    except Exception as e:
        print(f"An error occurred for post {post_id}: {e}")
        return (post_id, post_prompt, "Error")

# 主异步函数，处理所有博文
async def process_posts(sample):
    results_df = pd.DataFrame(columns=["博文", "判别结果", "id"])
    na_df = []
    ad_df = []

    with ThreadPoolExecutor(max_workers=39) as executor:
        loop = asyncio.get_event_loop()
        tasks = []
        for index, post in sample.iterrows():
            post_id = post['id']  
            # post_text=post['博文内容']
            post_prompt = f"{prompt_learning}请根据以上标准，对以下博文进行香港、广东人的判别：\n{post['博文内容']}\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅输出“香港：概率值；广东：概率值”即可"
            task = loop.create_task(run_in_executor(executor, call_api_sync, client, post_prompt, post_id))
            tasks.append(task)
        
        responses = await asyncio.gather(*tasks)
        for post_id, post_prompt, response in responses:
            if response not in ["No response", "Error"]:
                results_df = pd.concat([results_df, pd.DataFrame({'博文': [post_prompt], '判别结果': [response], 'id': [post_id]})], ignore_index=True)
                if response == "1":
                    ad_df.append(post_id)
            else:
                na_df.append(post_id)

    results_df.to_csv('glm-3_gd_diagnosis_temperature0.5.csv')
    print(results_df.head()) 

In [74]:
sample = sampled_gdip.iloc[150:200]

In [ ]:
loop = asyncio.get_event_loop()

if not loop.is_running():
    loop.run_until_complete(process_posts(sample))
else:
    task = loop.create_task(process_posts(sample))


In [26]:
# 4. 处理数据
sampled_gdip_100=pd.read_csv('glm-3_hk_diagnosis_gdip.csv')
sampled_hkip_100=pd.read_csv('glm-3_hk_diagnosis_hkip.csv')

In [28]:
def del_prompt_after_score(df):
    pattern = re.compile(r"\n香港人和广东人、其他大陆人的语言风格、词汇使用、语法结构、话题内容、地理标记不同，请仔细区分。处理香港人文本时，注重港式粤语词汇、特殊用语及英文混杂的特点。处理广东人文本时，注意广州粤语与香港粤语的区别。请分别输出帖子中香港人、广东人概率，输出概率值（0-100）。例如：帖子：“請問北上大陆的深圳買计数器和生果，可以過到關回香港嗎，唔該，thanks”。 分析： “過到關”粤语词汇， “北上”“大陆”“计数器”“生果”港人常用词汇，涉及香港的情况，粤语夹杂了英语；所以香港人概率90%返回格式：“香港：90%；广东10%；", re.MULTILINE)
    df['博文'] = df['博文'].apply(lambda x: re.sub(pattern, "", x))

    escaped_text = re.escape(prompt_learning)
    text_to_delete_1="请根据以上标准，对以下博文进行香港、广东人的判别："
    text_to_delete_2="请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅输出“香港：概率值；广东：概率值”即可"
    escaped_text_1=re.escape(text_to_delete_1)
    escaped_text_2=re.escape(text_to_delete_2)
    df['博文']=df['博文'].str.replace(escaped_text,"",regex=True)
    df['博文']=df['博文'].str.replace(escaped_text_1,"",regex=True)
    df['博文']=df['博文'].str.replace(escaped_text_2,"",regex=True)
    df['博文']=df['博文'].str.replace("\n","",regex=False)
    df['博文']=df['博文'].str.replace("?","",regex=False)
    df=df
    return df 


In [ ]:
gd_100_clean=del_prompt_after_score(sampled_gdip_100)
print(gd_100_clean.head())

In [ ]:
gdip_result = pd.merge(data_weibo_gd,gd_100_clean, on='id',how='inner')
hk_100_clean=del_prompt_after_score(sampled_hkip_100)
hkip_result = pd.merge(data_weibo_hk,hk_100_clean, on='id',how='inner')

In [39]:
hkip_result.to_csv('hkip_result.csv')
gdip_result.to_csv('gdip_result.csv')

In [ ]:
hkip_result=hkip_result[hkip_result['判别结果'].str.contains('香港|广东')]
print(len(hkip_result))
split_hkip_result=hkip_result['判别结果'].str.split('；|;',expand=True)
print(split_hkip_result)
hkip_result['香港概率'],hkip_result['广东概率']=split_hkip_result[0],split_hkip_result[1]
print(hkip_result.head())

In [ ]:
hkip_result.loc[98,'香港概率']='30%'
hkip_result.loc[98,'广东概率']='70%'
hkip_result['香港概率'] = hkip_result['香港概率'].str.replace('[^0-9.%]', '', regex=True)
hkip_result['广东概率'] = hkip_result['广东概率'].str.replace('[^0-9.%]', '', regex=True)
gdip_result=gdip_result[gdip_result['判别结果'].str.contains('香港|广东')]
print(len(gdip_result))
split_gdip_result=gdip_result['判别结果'].str.split('；|;',expand=True)
print(split_gdip_result)
gdip_result['香港概率'],gdip_result['广东概率']=split_gdip_result[0],split_gdip_result[1]
print(hkip_result.head())
gdip_result['广东概率'] = gdip_result['广东概率'].str.replace('[^0-9.%]', '', regex=True)
gdip_result['香港概率'] = gdip_result['香港概率'].str.replace('[^0-9.%]', '', regex=True)

In [ ]:
gdip_result['香港概率']  = gdip_result['香港概率'] .replace('0', np.nan)
gdip_result['广东概率']  = gdip_result['广东概率'] .replace('0', np.nan)
# gdip_result['香港概率'] = gdip_result['香港概率'].str.rstrip('%').astype(float)
# gdip_result['广东概率'] = gdip_result['广东概率'].str.rstrip('%').astype(float)
gdip_result['香港概率'] = pd.to_numeric(gdip_result['香港概率'].str.rstrip('%'), errors='coerce')
gdip_result['广东概率'] = pd.to_numeric(gdip_result['广东概率'].str.rstrip('%'), errors='coerce')
# 计算平均值
average_percentage_hk = gdip_result['香港概率'].mean()
average_percentage_gd = gdip_result['广东概率'].mean()
# 将平均值转换回百分数格式（如果需要）
average_percentage_str_hk = f"{average_percentage_hk}%"
average_percentage_str_gd = f"{average_percentage_gd}%"
print("广东ip中，香港概率平均为：",average_percentage_str_hk)
print("广东ip中，广东概率平均为：",average_percentage_str_gd)

# 2. 三步方法香港人判别

## 2.1 Step1:判断普通话和粤语

In [86]:
weibo_loc_space_analysis_result=pd.read_csv('weibo_loc_result_未分列.csv')

In [131]:
client = ZhipuAI(api_key="15a309caa3a78adf440c8072280fad02.ng01HdMeHCOodPUj")
prompt_learning="""
你是语言分析专家，判断并输出撰文者讲的是粤语和普通话的概率分别是多少，并解释分析过程（概率相加为1）。
注意：（1）词语方面：粤语较多保留了古汉语的特征，如“翼”。单音节词比普通话多，如“脷”、 复音词的词素次序与普通话有些不同，如“挤拥”、“齐整”。外来词比其他方言多，如“波”（ball）、“恤衫”（shirt）、“士多”（store）；有特有词，如“嘢”、“点解”、“巴闭”“睇”、古壮语“谂”等。（2）语法方面：粤语语法有修饰语后置的现象，如粤语“你行先”；粤语有回复体（叹翻下冷气）、始续体（食开饭就唔好睇报纸啦），比较句“我大过你”，表给予义的双宾 句“畀本书我”。常将形容词放在名词之后作修饰成分的语法结构，如“水大”“人客”“鱼生”等。（3）如果粤语与普通话的表达一样的情况下，默认为普通话。
例子：今日「衬」大陆银行开门…瞓醒又单拖出发北上.今日在莲塘口岸过关唔多人. 
输出格式：粤语：0.20，普通话：0.80，解释过程 
例子：按导航走走错出口（地图真的不靠谱）
输出格式：粤语：0.20，普通话：0.80，解释过程
"""
# 异步运行同步函数的辅助函数
async def run_in_executor(executor, func, *args):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(executor, func, *args)

# 进行API调用的同步函数
def call_api_sync(client, post_prompt, post_id):
    try:
        glm_completion = client.chat.completions.create(
            model="glm-3-turbo",
            messages=[{"role": "user", "content": post_prompt}],
            max_tokens=30,
        )
        response = glm_completion.choices[0].message.content if glm_completion.choices else "No response"
        return (post_id, post_prompt, response)
    except Exception as e:
        print(f"An error occurred for post {post_id}: {e}")
        return (post_id, post_prompt, "Error")

# 主异步函数，处理所有博文
async def process_posts(sample):
    results_df = pd.DataFrame(columns=["博文", "粤语识别结果", "id"])
    na_df = []
    ad_df = []

    with ThreadPoolExecutor(max_workers=40) as executor:
        loop = asyncio.get_event_loop()
        tasks = []
        for index, post in sample.iterrows():
            post_id = post['id']  
            post_prompt = f"{prompt_learning}请对以下博文进行粤语和普通话判别：\n{post['博文']}\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅按照要求输出识别结果即可"
            task = loop.create_task(run_in_executor(executor, call_api_sync, client, post_prompt, post_id))
            tasks.append(task)
        
        responses = await asyncio.gather(*tasks)
        for post_id, post_prompt, response in responses:
            if response not in ["No response", "Error"]:
                results_df = pd.concat([results_df, pd.DataFrame({'博文': [post_prompt], '粤语识别结果': [response], 'id': [post_id]})], ignore_index=True)
                if response == "1":
                    ad_df.append(post_id)
            else:
                na_df.append(post_id)

    results_df.to_csv('glm-3_weibo_loc_after_analysis_hk_diag_step1_test_4.csv')
    print(results_df.head()) 

In [ ]:
print(weibo_loc_space_analysis_result.columns)
sample = weibo_loc_space_analysis_result[(weibo_loc_space_analysis_result['id'] >=5001) & (weibo_loc_space_analysis_result['id'] <= 6000)]
print(len(sample))

In [ ]:
loop = asyncio.get_event_loop()

if not loop.is_running():
    loop.run_until_complete(process_posts(sample))
else:
    task = loop.create_task(process_posts(sample))


In [ ]:
df=pd.read_csv('glm-3_weibo_loc_after_analysis_hk_diag_step1_test_4.csv')
prompt_learning="""
你是语言分析专家，判断并输出撰文者讲的是粤语和普通话的概率分别是多少，并解释分析过程（概率相加为1）。
注意：（1）词语方面：粤语较多保留了古汉语的特征，如“翼”。单音节词比普通话多，如“脷”、 复音词的词素次序与普通话有些不同，如“挤拥”、“齐整”。外来词比其他方言多，如“波”（ball）、“恤衫”（shirt）、“士多”（store）；有特有词，如“嘢”、“点解”、“巴闭”“睇”、古壮语“谂”等。（2）语法方面：粤语语法有修饰语后置的现象，如粤语“你行先”；粤语有回复体（叹翻下冷气）、始续体（食开饭就唔好睇报纸啦），比较句“我大过你”，表给予义的双宾 句“畀本书我”。常将形容词放在名词之后作修饰成分的语法结构，如“水大”“人客”“鱼生”等。（3）如果粤语与普通话的表达一样的情况下，默认为普通话。
例子：今日「衬」大陆银行开门…瞓醒又单拖出发北上.今日在莲塘口岸过关唔多人. 
输出格式：粤语：0.20，普通话：0.80，解释过程 
例子：按导航走走错出口（地图真的不靠谱）
输出格式：粤语：0.20，普通话：0.80，解释过程
"""
escaped_text = re.escape(prompt_learning)
text_to_delete_1="请对以下博文进行粤语和普通话判别："
text_to_delete_2="\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅按照要求输出识别结果即可"
escaped_text_1=re.escape(text_to_delete_1)
escaped_text_2=re.escape(text_to_delete_2)

df['博文']=df['博文'].str.replace(escaped_text,"",regex=True)
df['博文']=df['博文'].str.replace(escaped_text_1,"",regex=True)
df['博文']=df['博文'].str.replace(escaped_text_2,"",regex=True)
df['博文']=df['博文'].str.replace("\n","",regex=True)
print(df.head())

In [ ]:
df=df[df['粤语识别结果'].str.contains('粤语|普通话')]
print(len(df))
split_df=df['粤语识别结果'].str.split('：|；|;|，|、',expand=True)
print(split_df.head())

In [ ]:
df['粤语概率'],df['普通话概率']=split_df[1],split_df[3]
df.to_csv('glm-3_weibo_loc_after_analysis_hk_diag_step1_497.csv')
df['粤语概率']=pd.to_numeric(df['粤语概率'],errors='coerce')
weibo_loc_after_step1=df[df['粤语概率']>=0.5]
weibo_loc_after_step1_belowhalf=df[df['粤语概率']<0.5]
print(len(weibo_loc_after_step1))# 271/
weibo_loc_after_step1.to_csv('weibo_loc_after_step1_test.csv')
weibo_loc_after_step1_belowhalf.to_csv('weibo_loc_after_step1_belowhalf.csv')


## 2.2 Step2:判断港式粤语和广式粤语

In [24]:
client = ZhipuAI(api_key="15a309caa3a78adf440c8072280fad02.ng01HdMeHCOodPUj")
prompt_learning="""
香港和广东均使用粤语，但语言风格、词汇使用、语法结构、话题内容、地理标记不同，请仔细区分。处理香港粤语文本时，注重港式粤语词汇、特殊用语及英文混杂的特点。处理广东粤语文本时，注意广式粤语与港式粤语的区别。请判断并输出社交媒体文本所用为港式粤语和广式粤语的概率（概率取值为0～1，不能为0.5，港式粤语和广式粤语概率和为1）
例1：“請問北上大陆的深圳買计数器和生果，可以過到關回香港嗎，唔該，thanks”。 
分析： “過到關”粤语词汇， “北上”“大陆”“计数器”“生果”港人常用词汇，涉及香港的情况，粤语夹杂了英语
输出格式：港式粤语：0.90，广式粤语：0.10
"""
# 异步运行同步函数的辅助函数
async def run_in_executor(executor, func, *args):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(executor, func, *args)

# 进行API调用的同步函数
def call_api_sync(client, post_prompt, post_id):
    try:
        glm_completion = client.chat.completions.create(
            model="glm-3-turbo",
            messages=[{"role": "user", "content": post_prompt}],
            temperature=0.5
        )
        response = glm_completion.choices[0].message.content if glm_completion.choices else "No response"
        return (post_id, post_prompt, response)
    except Exception as e:
        print(f"An error occurred for post {post_id}: {e}")
        return (post_id, post_prompt, "Error")

# 主异步函数，处理所有博文
async def process_posts(sample):
    results_df = pd.DataFrame(columns=["博文", "港式广式识别结果", "id"])
    na_df = []
    ad_df = []

    with ThreadPoolExecutor(max_workers=40) as executor:
        loop = asyncio.get_event_loop()
        tasks = []
        for index, post in sample.iterrows():
            post_id = post['id']  
            post_prompt = f"{prompt_learning}请根据以上评分标准，对以下博文进行港式粤语和广式粤语判别：\n{post['博文']}\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅按照要求输出识别结果即可"
            task = loop.create_task(run_in_executor(executor, call_api_sync, client, post_prompt, post_id))
            tasks.append(task)
        
        responses = await asyncio.gather(*tasks)
        for post_id, post_prompt, response in responses:
            if response not in ["No response", "Error"]:
                results_df = pd.concat([results_df, pd.DataFrame({'博文': [post_prompt], '港式广式识别结果': [response], 'id': [post_id]})], ignore_index=True)
                if response == "1":
                    ad_df.append(post_id)
            else:
                na_df.append(post_id)

    results_df.to_csv('glm-3_weibo_loc_after_analysis_hk_diag_step2.csv')
    print(results_df.head()) 


In [ ]:
# 筛选文本
print(weibo_loc_after_step1.columns)
sample = weibo_loc_after_step1.copy()
print(sample)

In [ ]:
loop = asyncio.get_event_loop()

if not loop.is_running():
    loop.run_until_complete(process_posts(sample))
else:
    task = loop.create_task(process_posts(sample))


In [ ]:
df=pd.read_csv('glm-3_weibo_loc_after_analysis_hk_diag_step2.csv')
prompt_learning="""
香港和广东均使用粤语，但语言风格、词汇使用、语法结构、话题内容、地理标记不同，请仔细区分。处理香港粤语文本时，注重港式粤语词汇、特殊用语及英文混杂的特点。处理广东粤语文本时，注意广式粤语与港式粤语的区别。请判断并输出社交媒体文本所用为港式粤语和广式粤语的概率（概率取值为0～1，不能为0.5，港式粤语和广式粤语概率和为1）
例1：“請問北上大陆的深圳買计数器和生果，可以過到關回香港嗎，唔該，thanks”。 
分析： “過到關”粤语词汇， “北上”“大陆”“计数器”“生果”港人常用词汇，涉及香港的情况，粤语夹杂了英语
输出格式：港式粤语：0.90，广式粤语：0.10
"""
escaped_text = re.escape(prompt_learning)
text_to_delete_1="请根据以上评分标准，对以下博文进行港式粤语和广式粤语判别：\n"
text_to_delete_2="\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅按照要求输出识别结果即可"
escaped_text_1=re.escape(text_to_delete_1)
escaped_text_2=re.escape(text_to_delete_2)

df['博文']=df['博文'].str.replace(escaped_text,"",regex=True)
df['博文']=df['博文'].str.replace(escaped_text_1,"",regex=True)
df['博文']=df['博文'].str.replace(escaped_text_2,"",regex=True)
df['博文']=df['博文'].str.replace("\n","",regex=True)
df=df[df['港式广式识别结果'].str.contains('港式粤语|广式粤语')]
print(len(df))
split_df=df['港式广式识别结果'].str.split('：|；|;|，',expand=True)
print(split_df.head())


In [ ]:
print(df['港式广式识别结果'])
split_df=df['港式广式识别结果'].str.split('：|，',expand=True)
print(split_df.head())

In [ ]:
df['港式粤语概率'],df['广式粤语概率']=split_df[1],split_df[3]
print(df.head())
df['港式粤语概率']=pd.to_numeric(df['港式粤语概率'],errors='coerce')
weibo_loc_after_step2=df[df['港式粤语概率']>=0.5]
print(len(weibo_loc_after_step2))# 254


In [ ]:
weibo_loc_after_step1_step2=pd.merge(weibo_loc_after_step1,weibo_loc_after_step2,on='id',how='inner')
print(weibo_loc_after_step1_step2.head())
weibo_loc_after_step1_step2=weibo_loc_after_step1_step2.drop(['Unnamed: 0_x','Unnamed: 0_y','博文_y'],axis=1)
weibo_loc_after_step1_step2.to_csv('weibo_loc_after_step1_step2.csv')
weibo_loc_after_step1_2=pd.merge(weibo_loc_after_step1_step2,weibo_loc_space_analysis_result,on='id',how='inner')
print(weibo_loc_after_step1_2.head())
weibo_loc_after_step1_2=weibo_loc_after_step1_2.drop(['博文','博文内容','Unnamed: 0.1'],axis=1)
weibo_loc_after_step1_2.to_csv('weibo_loc_after_step1_2_包含打分.csv')

## 2.3 Step3:粤语概率小于50%，判断地理位置

In [61]:
client = ZhipuAI(api_key="15a309caa3a78adf440c8072280fad02.ng01HdMeHCOodPUj")
prompt_learning="""
判断这段话的语义是否包含立足香港的话题，包含为1，不包含为0。输出0或1。
1代表：将出发地或居住地定为香港（北上深圳、住屯门等香港地名），以及将返回地定为香港（返港等）、对比香港和深圳并提出香港问题的句子；
0代表：去香港或回深圳等隐含将深圳作为居住地、不包含立足香港的话题
注意：不要与深圳人去香港的帖子混淆
例子：周末在邊境，下周要不回深圳
输出格式：0
"""
# 异步运行同步函数的辅助函数
async def run_in_executor(executor, func, *args):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(executor, func, *args)

# 进行API调用的同步函数
def call_api_sync(client, post_prompt, post_id):
    try:
        glm_completion = client.chat.completions.create(
            model="glm-3-turbo",
            messages=[{"role": "user", "content": post_prompt}],
            temperature=0.5
        )
        response = glm_completion.choices[0].message.content if glm_completion.choices else "No response"
        return (post_id, post_prompt, response)
    except Exception as e:
        print(f"An error occurred for post {post_id}: {e}")
        return (post_id, post_prompt, "Error")

# 主异步函数，处理所有博文
async def process_posts(sample):
    results_df = pd.DataFrame(columns=["博文", "立足香港识别结果", "id"])
    na_df = []
    ad_df = []

    with ThreadPoolExecutor(max_workers=40) as executor:
        loop = asyncio.get_event_loop()
        tasks = []
        for index, post in sample.iterrows():
            post_id = post['id']  
            post_prompt = f"{prompt_learning}请根据以上评分标准，对以下博文进行用户是否立足香港判别：\n{post['博文']}\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅按照要求输出识别结果即可"
            task = loop.create_task(run_in_executor(executor, call_api_sync, client, post_prompt, post_id))
            tasks.append(task)
        
        responses = await asyncio.gather(*tasks)
        for post_id, post_prompt, response in responses:
            if response not in ["No response", "Error"]:
                results_df = pd.concat([results_df, pd.DataFrame({'博文': [post_prompt], '立足香港识别结果': [response], 'id': [post_id]})], ignore_index=True)
                if response == "1":
                    ad_df.append(post_id)
            else:
                na_df.append(post_id)

    results_df.to_csv('glm-3_weibo_loc_after_analysis_hk_diag_step3_2.csv')
    print(results_df.head()) 


In [ ]:
# 筛选文本
print(weibo_loc_after_step1_belowhalf.columns)
sample = weibo_loc_after_step1_belowhalf.copy()
print(sample)



In [ ]:
loop = asyncio.get_event_loop()

if not loop.is_running():
    loop.run_until_complete(process_posts(sample))
else:
    task = loop.create_task(process_posts(sample))

In [65]:
df=pd.read_csv('glm-3_weibo_loc_after_analysis_hk_diag_step3_2.csv')
prompt_learning="""
判断这段话的语义是否包含立足香港的话题，包含为1，不包含为0。输出0或1。
1代表：将出发地或居住地定为香港（北上深圳、住屯门等香港地名），以及将返回地定为香港（返港等）、对比香港和深圳并提出香港问题的句子；
0代表：去香港或回深圳等隐含将深圳作为居住地、不包含立足香港的话题
注意：不要与深圳人去香港的帖子混淆
例子：周末在邊境，下周要不回深圳
输出格式：0
"""
escaped_text = re.escape(prompt_learning)
text_to_delete_1="请根据以上评分标准，对以下博文进行用户是否立足香港判别：\n"
text_to_delete_2="\n请您分析,不需要给出如‘好的，我会按照您的要求输出’和‘请注意’等类似的额外说明，仅按照要求输出识别结果即可"
escaped_text_1=re.escape(text_to_delete_1)
escaped_text_2=re.escape(text_to_delete_2)

df['博文']=df['博文'].str.replace(escaped_text,"",regex=True)
df['博文']=df['博文'].str.replace(escaped_text_1,"",regex=True)
df['博文']=df['博文'].str.replace(escaped_text_2,"",regex=True)
df['博文']=df['博文'].str.replace("\n","",regex=True)


In [ ]:
df['立足香港识别结果']=df['立足香港识别结果'].apply(lambda x:str(x))
df_hk=df[(df['立足香港识别结果'].str.contains('0').fillna(False))|(df['立足香港识别结果']=='0')]
print(df_hk)